# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [1]:
import findspark
findspark.init()
import ta
from foraneos.utils_proyectofinal_copy import resample_and_aggregate
from foraneos.utils_proyectofinal_copy import SparkUtils as SpU

## Spark Session creation


In [2]:
from pyspark.sql import SparkSession

SPARK_SERVER = {'Konrad': '672e28abb623',
                'Aaron' : 'a5ab6bdab4b3'}
KAFKA_SERVER = {'Konrad': 'dee5c9cc3710:9093',
                'Aaron' : '69b1b3611d90:9093'}
current_user = 'Konrad'

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://{}:7077".format(SPARK_SERVER[current_user])) \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
    
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94156cee-3332-429b-821c-724b265d521f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

## Kafka Stream creation

In [3]:
streamer_lines = []

for i in range(4):
    streamer_lines.append( spark \
                            .readStream \
                            .format("kafka") \
                            .option("kafka.bootstrap.servers", "{}:9093".format(KAFKA_SERVER[current_user])) \
                            .option("subscribe", f"stock_topic{i}") \
                            .load()
    )


In [4]:
result_schema = SpU.generate_schema([("Timestamp", "timestamp" ), 
                                              ("Company", "string" ),
                                              ("Open", "float" ), 
                                              ("high", "float" ), 
                                              ("Low", "float"),
                                              ("Close", "float" ) ,
                                              ("williams_r", "float" ),  
                                              ("rsi", "float"), 
                                              ("ultimate_osc", "float"), 
                                              ("close_lag_1", "float" ), 
                                              ("close_lag_2", "float" ), 
                                              ("close_lag_3", "float" ), 
                                              ("close_lag_4", "float" ), 
                                              ("close_lag_5", "float" ),                                                                                 
                                              ])





## Transform binary data into string

In [5]:
from pyspark.sql.functions import col, split
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.sql.functions import pandas_udf

streamer_df = []

for i in range(4):
       
           
    df = streamer_lines[i].withColumn("value_str", col("value").cast("string"))
    df = df.withColumn("split", split(col("value_str"), ","))
    df = df.withColumn("timestamp", col("split").getItem(0).cast(TimestampType())) \
           .withColumn("company", col("split").getItem(1)) \
           .withColumn("close", col("split").getItem(2).cast(DoubleType())) \
          .select("timestamp", "company","close")
          
    custom_resampler = resample_and_aggregate(new_window=5)
    resampled_df = df.mapInPandas(custom_resampler, schema=result_schema)

    streamer_df.append(resampled_df)



### Sink configuration

In [ ]:
query = []

for i in range(4):
    query.append(
        streamer_df[i] \
        .writeStream \
        .outputMode("append") \
        .trigger(processingTime='30 seconds') \
        .format("parquet") \
        .option("path", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/output{i}/")
        .option("checkpointLocation", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/checkpoints/stock_topic{i}") \
        .start()
    )

    #query[i].awaitTermination(100)


25/05/12 22:05:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 22:05:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 22:05:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 22:05:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/05/12 22:05:26 WARN ClientUtils: Couldn't resolve server dee5c9cc3710:9093:9093 from bootstrap.servers as DNS resolution failed for dee5c9cc3710:9093
25/05/12 22:05:26 WARN ClientUtils: Couldn't resolve server dee5c9cc3710:9093:9093 from bootstrap.servers as DNS resolution failed for dee5c9cc3710:9093
25/05/12 22:05:26 WARN ClientUtils: Couldn't resolve server dee5c9cc3710:9093:9093 from bootstrap.servers as DNS resolution failed for dee5c9cc3710:9093
25/05/12 22:05:26 WARN ClientUtils: Couldn't resolve server dee5c9cc3710:9093:9093 from bootstrap.servers as DNS resolution failed for dee5c9cc3710:9093
25/05/12 22:05:26 WARN KafkaOffsetReaderAdmin: Error in attempt 1 getting Kafka offsets: 
org.apache.kafka.common.KafkaException: Failed to create new KafkaAdminClient
	at org.apache.kafka.clients.admin.KafkaAdminClient.createInternal(KafkaAdminClient.java:551)
	at org.apache.kafka.clients.admin.Admin.create(Admin.java:144)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.createAdmin

In [7]:
for i in range(4):
    query[i].stop()

In [8]:
sc.stop()